# Tristan notebook

In [ ]:
import pandas as pd

In [ ]:
X_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(X_train_path)

In [ ]:
X_train

In [ ]:
X_train["piezo_station_bss_id"].nunique()

In [ ]:
X_train["piezo_groundwater_level_category"].unique()

In [ ]:
# dictionary to encode the target
target_cat = {'High':4, 'Very High':5, 'Very Low' :1, 'Low':2, 'Average':3}
target_level = {num: level for level, num in target_cat.items()}
target_level[0] = 'NaN'

## Station locations

In [ ]:
stations_ids = X_train[["piezo_station_bss_id", "piezo_station_latitude", "piezo_station_longitude", "piezo_groundwater_level_category"]].drop_duplicates()
stations_coords = X_train[["piezo_station_bss_id", "piezo_station_latitude", "piezo_station_longitude"]].drop_duplicates()
stations_coords.index = stations_coords["piezo_station_bss_id"]
stations_coords = stations_coords.drop(columns=['piezo_station_bss_id'])
stations_coords.head()

In [ ]:
stations_ids["level"] = stations_ids["piezo_groundwater_level_category"].apply(lambda x : target_cat.get(x, 0))

In [ ]:
%pip install cartopy


In [ ]:
# cartopy to display maps
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
def display_piezzo_stations(color_column=None):
    """Plot the weather station on a map of Europe"""
    # Load coordinates from other notebook

    fig = plt.figure( figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([-5, 10, 42, 52], crs=ccrs.PlateCarree())

    # Draw the background
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    # ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.RIVERS)
    ax.gridlines(draw_labels=True)

    # Add stations' names and positions
    if color_column is None:
        stations_coords.plot.scatter(x="piezo_station_longitude", y="piezo_station_latitude", s=4, ax=ax, transform=ccrs.PlateCarree())
    else:
        stations_coords.plot.scatter(x="piezo_station_longitude", y="piezo_station_latitude", c=color_column, cmap='tab20' ,s=4, ax=ax, transform=ccrs.PlateCarree())

    ax.set_title("Piezzo stations locations")

    plt.show()

In [ ]:
display_piezzo_stations()

## Covariance of stations

In [ ]:
# subset of the stations
df_by_station=  X_train[["piezo_station_bss_id", "piezo_measurement_date", "piezo_groundwater_level_category"]]
print(df_by_station.shape)
print(df_by_station.index)


In [ ]:
# create the level column
df_by_station["level"] = df_by_station["piezo_groundwater_level_category"].apply(lambda x : target_cat.get(x, 0))
df_by_station.head()


In [ ]:
# pivot to get the time series
df_by_station = df_by_station.pivot_table(index= "piezo_measurement_date",columns="piezo_station_bss_id", values="level")
df_by_station = df_by_station.fillna(value=0)
print(df_by_station.shape)
df_by_station.head()

In [ ]:
# plot the evolution over time of the first 10 stations
fig, ax = plt.subplots(1, 1, figsize=(20, 6))
df_by_station.loc[:, df_by_station.columns[:10]].plot(ax=ax)
ax.legend(title="station", fontsize="small")
ax.set_title("Ground Water level evolution over time in the train dataset (10 stations)")
ax.set_ylabel("level")
ax.set_yticks([i for i in range(6)], labels=[target_level[i] for i in range(6)])
plt.show()

In [ ]:
df_by_station.to_numpy()

In [ ]:
# Find clusters
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.00000000001, min_samples=100)
clustering.fit(df_by_station.to_numpy())
print(clustering.labels_)
max(clustering.labels_)




In [ ]:
from sklearn.cluster import k_means
centroid, label, intertia = k_means(df_by_station.T, n_clusters=20, random_state=1, n_init=10)
max(label)



In [ ]:
display_piezzo_stations(label)

In [ ]:
centroid.shape

In [ ]:
# plot the evolution over time of the first 10 stations
fig, ax = plt.subplots(2, 1, figsize=(20, 8))
ax[0].plot(centroid[:10].T)
#ax[0].legend(title="Centroid station", fontsize="small")
ax[0].set_title("Ground Water level evolution over time in the train dataset (0-9 centroid stations)")
ax[0].set_ylabel("level")
ax[0].set_yticks([i for i in range(6)], labels=[target_level[i] for i in range(6)])

ax[1].plot(centroid[10:].T)
ax[1].set_title("Ground Water level evolution over time in the train dataset (10-19 centroid stations)")
ax[1].set_ylabel("level")
ax[1].set_yticks([i for i in range(6)], labels=[target_level[i] for i in range(6)])
plt.show()

In [ ]:
#from sklearn.model_selection import TimeSeriesSplit
#tss = TimeSeriesSplit(n_splits=5)  # Deflaut values are ok
# for i, (train_index, test_index) in enumerate(tss.split(df_by_station)):

## Stations


In [ ]:
X_train[["piezo_station_bss_id","piezo_station_investigation_depth"]].drop_duplicates()

## Feature creation:
- past month precipitation `meteo_rain_heigh` + `meteo_snow_heigh` - `meteo_evatranspiration_grid` [mm/Day]
- past year precitipitation
- neighboring cities 
meteo_evapotranspiration_grid, meteo_radiation_UV, 
## Investigation
- distance_piezo_meteo
- distance_hydro_meteo

In [ ]:
station_df = X_train[X_train["piezo_station_bss_id"] == "BSS000ACKJ"][["piezo_measurement_date", "meteo_rain_height", "meteo_snow_height" , "meteo_evapotranspiration_grid"]]
station_df["piezo_measurement_date"] =  pd.to_datetime(station_df["piezo_measurement_date"])
station_df = station_df.set_index("piezo_measurement_date")
station_df = station_df.fillna(0.0)
# Calculate the rolling sum for the previous month (30 days)
station_df['precipitation_30d_sum'] = station_df['meteo_rain_height'].rolling(window=30, min_periods=1).sum()
station_df['snow_30d_sum'] = station_df['meteo_snow_height'].rolling(window=30, min_periods=1).sum()
station_df['evapotranspiration_30d_sum'] = station_df['meteo_evapotranspiration_grid'].rolling(window=30, min_periods=1).sum()
station_df["bilan_30d"] = station_df["precipitation_30d_sum"] + station_df["snow_30d_sum"] - station_df["evapotranspiration_30d_sum"]
# Calculate the rolling sum for the previous month (90 days)
station_df['precipitation_90d_sum'] = station_df['meteo_rain_height'].rolling(window=30, min_periods=1).sum()
station_df['snow_90d_sum'] = station_df['meteo_snow_height'].rolling(window=90, min_periods=1).sum()
station_df['evapotranspiration_90d_sum'] = station_df['meteo_evapotranspiration_grid'].rolling(window=90, min_periods=1).sum()
# Calculate the rolling sum for the past year (365 days)
station_df['precipitation_365d_sum'] = station_df['meteo_rain_height'].rolling(window=365, min_periods=1).sum()
station_df['snow_365d_sum'] = station_df['meteo_snow_height'].rolling(window=365, min_periods=1).sum()
station_df['evapotranspiration_365d_sum'] = station_df['meteo_evapotranspiration_grid'].rolling(window=365, min_periods=1).sum()
station_df

In [ ]:
# Display the evolution of bilans
fig, ax = plt.subplots(4, 1, figsize=(20, 12), layout="constrained")
daily = ["meteo_rain_height", "meteo_snow_height", "meteo_evapotranspiration_grid"]
monthly = ['precipitation_30d_sum', 'snow_30d_sum', 'evapotranspiration_30d_sum']
trimester = ['precipitation_90d_sum', 'snow_90d_sum', 'evapotranspiration_90d_sum']
yearly =  ['precipitation_365d_sum', 'snow_365d_sum', 'evapotranspiration_365d_sum']
import matplotlib.dates as mdates
line_width =1
# Daily
for var in daily:
    ax[0].plot(station_df[var], label=var, linewidth=line_width)
ax[0].legend()
ax[0].set_title("Daily")
ax[0].set_ylabel("Precipitation [mm]")
# Set the major locator to month and formatter to month names
ax[0].xaxis.set_major_locator(mdates.MonthLocator())
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%b%y'))
# Rotate the x-axis labels for better readability
plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Montly
for var in monthly:
    ax[1].plot(station_df[var], label=var, linewidth=line_width)
ax[1].legend()
ax[1].set_title("Montly cumul")
ax[1].set_ylabel("Precipitation [mm]")
# Set the major locator to month and formatter to month names
ax[1].xaxis.set_major_locator(mdates.MonthLocator())
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%b%y'))
# Rotate the x-axis labels for better readability
plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Montly
for var in trimester:
    ax[2].plot(station_df[var], label=var, linewidth=line_width)
ax[2].legend()
ax[2].set_title("Trimester cumul")
ax[2].set_ylabel("Precipitation [mm]")
# Set the major locator to month and formatter to month names
ax[2].xaxis.set_major_locator(mdates.MonthLocator())
ax[2].xaxis.set_major_formatter(mdates.DateFormatter('%b%y'))
# Rotate the x-axis labels for better readability
plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Yearly
for var in yearly:
    ax[3].plot(station_df[var], label=var, linewidth=line_width)
ax[3].legend()
ax[3].set_title("Yearly cumul")
ax[3].set_ylabel("Precipitation [mm]")
# Set the major locator to month and formatter to month names
ax[3].xaxis.set_major_locator(mdates.MonthLocator())
ax[3].xaxis.set_major_formatter(mdates.DateFormatter('%b%y'))
# Rotate the x-axis labels for better readability
plt.setp(ax[3].xaxis.get_majorticklabels(), rotation=45, ha='right')
plt.show()

In [ ]:
def add_bilan_days(station_df:pd.DataFrame, days:list[int])->pd.DataFrame:
    """Create a new column with the rolling water-bilan over d days.
    past d-days precipitation= `meteo_rain_heigh` + `meteo_snow_heigh` - `meteo_evatranspiration_grid` [mm/Day]
    """
    station_df["piezo_measurement_date"] =  pd.to_datetime(station_df["piezo_measurement_date"])
    station_df = station_df.set_index("piezo_measurement_date")
    station_df = station_df.fillna(0.0)
    for d in days:
        calc_df = pd.DataFrame(index=station_df.index)
        # Calculate the rolling sum d days)
        calc_df[f'precipitation_{d}d_sum'] = station_df['meteo_rain_height'].rolling(window=d, min_periods=1).sum()
        calc_df[f'snow_{d}d_sum'] = station_df['meteo_snow_height'].rolling(window=d, min_periods=1).sum()
        calc_df[f'evapotranspiration_{d}_sum'] = station_df['meteo_evapotranspiration_grid'].rolling(window=d, min_periods=1).sum()
        station_df[f"water_bilan_{d}d"] = calc_df[f"precipitation_{d}d_sum"] + calc_df[f"snow_{d}d_sum"] - calc_df[f"evapotranspiration_{d}_sum"]
    
    return station_df

In [ ]:
station_df = X_train[X_train["piezo_station_bss_id"] == "BSS000ACKJ"][["piezo_measurement_date", "meteo_rain_height", "meteo_snow_height" , "meteo_evapotranspiration_grid", "piezo_groundwater_level_category"]]
station_df["level"] = station_df["piezo_groundwater_level_category"].apply(lambda x : target_cat.get(x, 0))
station_df = add_bilan_days(station_df, [2, 3, 7, 10, 30, 90])
station_df

In [ ]:
# Plot the water level with the
fig, ax = plt.subplots(figsize=(20, 6), layout='constrained')
for var in station_df.columns:
    if 'bilan' in var:
        ax.plot(station_df.iloc[:800][var], label=var)
ax.set_ylabel("Bilan of water variation")
twin_x = ax.twinx()
twin_x.plot(station_df.iloc[:800]["level"], label="taget level", c='b', linestyle="dashed")
twin_x.set_yticks([i for i in range(6)], labels=[target_level[i] for i in range(6)], c='b')
ax.legend()
twin_x.legend()
twin_x.set_ylabel("Water ground target level", c='b')
ax.set_title("Correlation with the water ground level for station BSS000ACKJ")
plt.show()

In [ ]:
# Extract the time serie for each station
sub_X_train = X_train
station_df = pd.DataFrame()
base_df = pd.DataFrame()
for nb_stations, station_id in enumerate(sub_X_train["piezo_station_bss_id"]):
    base_df = sub_X_train[sub_X_train["piezo_station_bss_id"] == station_id]
    station_df = sub_X_train[sub_X_train["piezo_station_bss_id"] == station_id][["piezo_measurement_date", "meteo_rain_height", "meteo_snow_height" , "meteo_evapotranspiration_grid", "piezo_groundwater_level_category"]]

    station_df["level"] = station_df["piezo_groundwater_level_category"].apply(lambda x : target_cat.get(x, 0))
    station_df = add_bilan_days(station_df, [2, 3, 7, 10, 30, 90])
    
    # print(station_id)
    if nb_stations > 1 : 
        break

In [ ]:
station_df

In [ ]:
# Plot the water level with the water bilan
fig, ax = plt.subplots(figsize=(20, 6), layout='constrained')
for var in station_df.columns:
    if 'bilan' in var:
        ax.plot(station_df.iloc[:800][var], label=var)
ax.set_ylabel("Bilan of water variation")
twin_x = ax.twinx()
twin_x.plot(station_df.iloc[:800]["level"], label="taget level", c='b', linestyle="dashed")
twin_x.set_yticks([i for i in range(6)], labels=[target_level[i] for i in range(6)], c='b')
ax.legend()
twin_x.legend()
twin_x.set_ylabel("Water ground target level", c='b')
ax.set_title(f"Correlation with the water ground level for station {station_id}")
plt.show()

In [ ]:
cols_names = X_train.columns
print(", ".join(name for name in cols_names if "meteo" in name))

In [ ]:
X_train[["distance_piezo_meteo", "distance_piezo_hydro"]].mean()